In [ ]:
# Code adapted from: https://www.kaggle.com/ma7555/fisher-s-linear-discriminant-from-scratch-85-98
# Memory profiler: https://pypi.org/project/memory-profiler/

In [14]:
import project1 as p1
import utils

import numpy as np
#import pandas as pd
#from matplotlib import pyplot as plt

In [15]:
train_data = utils.load_data('reviews_train.tsv')
val_data = utils.load_data('reviews_val.tsv')
test_data = utils.load_data('reviews_test.tsv')

train_texts, train_labels = zip(*((sample['text'], sample['sentiment']) for sample in train_data))
val_texts, val_labels = zip(*((sample['text'], sample['sentiment']) for sample in val_data))
test_texts, test_labels = zip(*((sample['text'], sample['sentiment']) for sample in test_data))

dictionary = p1.bag_of_words(train_texts)

train_bow_features = p1.extract_bow_feature_vectors(train_texts, dictionary)
val_bow_features = p1.extract_bow_feature_vectors(val_texts, dictionary)
test_bow_features = p1.extract_bow_feature_vectors(test_texts, dictionary)

print("1")

1


In [20]:
im_train = train_bow_features
train_labels = train_labels

nclasses = len(np.unique(train_labels))
nfeatures = np.size(im_train, axis=1)
class_indexes = []
for i in range(nclasses):
    class_indexes.append(np.argwhere(train_labels == i))

print(im_train.shape)

(4000, 13234)


In [17]:
class_means, other_class_means = np.empty((nclasses, nfeatures)), np.empty((nclasses, nfeatures))
other_class = []
SW_one, SW_two, SW = np.zeros((nclasses, nfeatures, nfeatures)), np.zeros((nclasses, nfeatures, nfeatures)), np.zeros((nclasses, nfeatures, nfeatures))
W = np.zeros((nclasses, nfeatures, 1))
W0 = np.zeros((nclasses))

print("3")

3


In [9]:
for i in range(nclasses):
    class_means[i] = np.mean(im_train[class_indexes[i]], axis=0)
    other_class.append(np.delete(im_train, class_indexes[i], axis=0)) # one-versus-the-rest approach
    other_class_means[i] = np.mean(other_class[i], axis=0)
    between_class1 = np.subtract(im_train[class_indexes[i]].reshape(-1, nfeatures),
                                 class_means[i])
    SW_one[i] = between_class1.T.dot(between_class1)
    between_class2 = np.subtract(other_class[i], other_class_means[i])
    SW_two[i] = between_class2.T.dot(between_class2)
    SW[i] = SW_one[i] + SW_two[i]
    W[i] = np.dot(np.linalg.pinv(SW[i]),
                  np.subtract(other_class_means[i],
                              class_means[i]).reshape(-1, 1))
    W0[i] = -0.5 * np.dot(W[i].T, (class_means[i] + other_class_means[i]))
print("3")

/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


ValueError: could not broadcast input array from shape (0,13234) into shape (13234,)

In [18]:
print(SW.shape)
print(W.shape)
print(W[:10])
print(W0.shape)

print("3")

(2, 13234, 13234)
(2, 13234, 1)
[[[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]]
(2,)
3


In [ ]:
im_test = val_bow_features
im_test.shape

In [ ]:
Y = np.zeros((len(im_test), nclasses))
predict = np.zeros((len(im_test)), dtype=int)
for j in range(len(im_test)):
    for i in range(nclasses):
        Y[j, i] = np.dot(W[i].T,  im_test[j]) + W0[i]
    predict[j] = np.argmin(Y[j])

In [ ]:
predict[:10]

In [ ]:
accuracy = (predict == val_labels).mean()
print(accuracy)